In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[1]:

In [ ]:
get_ipython().magic('pylab nbagg')
from tvb.simulator.lab import *

Exploring The Bold Monitor<br>
===============================================================

This tutorial explores the different functions used to model the haemodynamic response function (HRF) to compute the BOLD (Blood Oxygenation Level Dependent) signal. <br>
<br>
In the current implementation (1.1.3) TVB has HRF kernels:<br>
<br>
1. a Gamma function,<br>
2. a difference of two damped oscillators, <br>
3. a first order Volterra kernel, and<br>
4. a mixture of gamma functions. 

*Bold Monitor*<br>
--------<br>
<br>
<br>
<br>
Let's start by creating an instance of the Bold monitor with its default parameters:

In[2]:

In [ ]:
bold = monitors.Bold()
bold.configure()
bold

In general, the sampling period of a monitor is in milliseconds and must be an integral multiple of the integration-step size used in a simulation. <br>
<br>
Therefore, monitors need to know the integration time step (*dt*) because some data reduction mechanims (eg, downsampling to the monitor's sampling period) depend on it. An easy way to achieve this is:

In[3]:

In [ ]:
bold.dt = 2**-4 # Default value used in the scripts found at tvb/simulator/demos

HRFs are TVB Equation datatypes, and you can explore their attributes, 

In[4]:

In [ ]:
bold.hrf_kernel

The default kernel is the **Volterra kernel**. The shape of this function depends on the following parameters: <br>
<br>
1. $\tau_s$, rate constant of signal decay;<br>
2. $\tau_f$, rate constant of feedback regulation;<br>
<br>
* $V_0$ and $k_1$ are parameters used in the monitor to scale the amplitude of the response. See [1]. 

Let's have a look at the function:

In[5]:

In [ ]:
bold.compute_hrf()

By default, the method **compute_hrf** gives the reflected version of the HRF. The product between this reflected HRF and the monitor's neural activity history (convolution) yields the BOLD signal. In python the indexing [::-1] will give the HRF kernel as often seen in scientific publications. 

In[6]:

plot the kernel

In [ ]:
plt.plot(bold._stock_time, bold.hemodynamic_response_function.T[::-1]); 
plt.ylabel('hrf');
plt.xlabel('time [sec]')

plot the maximum

In [ ]:
plt.plot(bold._stock_time[bold.hemodynamic_response_function.T[::-1].argmax()], bold.hemodynamic_response_function.T[::-1].max(), 'ko')

In[7]:

In [ ]:
print('Rising peak is around %1.2f seconds' % bold._stock_time[bold.hemodynamic_response_function.T[::-1].argmax()])

Exploring other HRF kernels<br>
---------------------------<br>
<br>
 <br>
First, we will create new instances of the Bold monitor.<br>
<br>
Second, the equation defining the hrf kernel has to be changed. To achieve this we will make use of the predefined functions as Equations datatypes <br>
<br>
- In [2] they used a simple gamma function; in [3] a difference of damped oscillators was fitted to functional data from the visual cortex; the mixture of gammas is the function used in softwares like SPM. 

In[8]:

In [ ]:
hrf_kernels = [equations.FirstOrderVolterra(),
               equations.DoubleExponential(),
               equations.MixtureOfGammas(),
               equations.Gamma()]

In [ ]:
figure()
for hrf in hrf_kernels: 
    bold_monitor = monitors.Bold(hrf_kernel=hrf)
    bold_monitor.dt = 2**-4
    bold_monitor.compute_hrf()
    plot(bold_monitor._stock_time,
         bold_monitor.hemodynamic_response_function.T[::-1], 
         label=hrf.__class__.__name__);

In [ ]:
ylabel('hrf');
xlabel('time [sec]')    
legend()

* [1] Friston, K., Mechelli, A., Turner, R., and Price, C., *Nonlinear Responses in fMRI: The Balloon Model, Volterra Kernels, and Other Hemodynamics*, NeuroImage, 12, 466 - 477, 2000. <br>
<br>
* [2] Geoffrey M. Boynton, Stephen A. Engel, Gary H. Glover and David J. Heeger (1996). *Linear Systems Analysis of Functional Magnetic Resonance Imaging in Human V1*. J Neurosci 16: 4207-4221<br>
<br>
* [3] Alex Polonsky, Randolph Blake, Jochen Braun and David J. Heeger (2000). *Neuronal activity in human primary visual cortex correlates with perception during binocular rivalry*. Nature Neuroscience 3: 1153-1159<br>
<br>
* [4] Glover, G. *Deconvolution of Impulse Response in Event-Related BOLD fMRI*. NeuroImage 9, 416-429, 1999.<br>
<br>
* [5] Have a look at this tutorial: http://nbviewer.ipython.org/github/practical-neuroimaging/pna-notebooks/blob/master/convolution.ipynb<br>
<br>
* [6] Drysdale, P. M.; Huber, J. P.; Robinson, P. A. & Aquino, K. M. *Spatiotemporal BOLD dynamics from a poroelastic hemodynamic model*. J Theor Biol, 2010, 265, 524–534<br>
<br>
* [7] http://en.wikibooks.org/wiki/SPM/Haemodynamic_Response_Function<br>
